<a href="https://colab.research.google.com/github/VICTORIA-OKESIPE/Data-Science-Competitions/blob/main/Team_Bambasylla_expresso_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Expresso Churn Prediction Challenge

#Team: Bambasylla

### Author 1: Victoria

### Author 2: Allassan

Creation date: 15/10/2021

Update: 09/11/2021

In [ ]:
# we use the zindi library to retreive information directly from the zindi website
!pip install -q git+https://github.com/eaedk/testing-zindi-package.git

In [ ]:
# Import Zindian class from the package
from zindi.user import Zindian

In [ ]:
# create a user object
my_username = "WedOO_ai"
user = Zindian(username = my_username)

Your password
>> ··········

[ 🟢 ] 👋🏾👋🏾 Welcome WedOO_ai 👋🏾👋🏾



In [ ]:
user.select_a_challenge()                               # Select a Zindi challenge
user.which_challenge                                    # Get information about the selected challenge

__________________________________________________________________________________________________________________________________
|     |              |                  |                    |          
|index|  challenge   |     problem      |       reward       |    id    
|     |              |                  |                    |          
----------------------------------------------------------------------------------------------------------------------------------
|  0  |Public Compet |  Classification  |     $5 000 USD     | kenyan-sign-language-classification-challenge...
----------------------------------------------------------------------------------------------------------------------------------
|  1  | Public Hack  |  Classification  |      $300 USD      | kenyan-sign-language-classification-hackathon...
----------------------------------------------------------------------------------------------------------------------------------
|  2  |Public Compet |  Classific

'expresso-churn-prediction'

In [ ]:
user.download_dataset(destination="dataset") # Download the dataset of the selected challenge

dataset/StarterNotebook.ipynb: 100%|██████████| 334k/334k [00:00<00:00, 501ko/s]
dataset/VariableDefinitions.csv: 100%|██████████| 1.41k/1.41k [00:00<00:00, 800ko/s]
dataset/SampleSubmission.csv: 100%|██████████| 15.6M/15.6M [00:02<00:00, 6.20Mo/s]
dataset/Test.csv: 100%|██████████| 42.9M/42.9M [00:06<00:00, 7.28Mo/s]
dataset/Train.csv: 100%|██████████| 247M/247M [00:52<00:00, 4.89Mo/s]


In [ ]:
#path to datasets:
path='dataset/'

In [ ]:
#importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,KFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.cluster import KMeans

In [ ]:
#chech over the different variables:
var_def=pd.read_csv(path+'VariableDefinitions.csv')

In [ ]:
var_def

,Variable Definitions,Unnamed: 1,Unnamed: 2
0,NaN,French,English
1,NaN,Le dataset churn comprend 19 variables dont 15...,The churn dataset includes 19 variables includ...
2,user_id,NaN,NaN
3,REGION,la localité de chaque client,the location of each client
4,TENURE,la durée dans le reseau,duration in the network
5,MONTANT,montant de recharge,top-up amount
6,FREQUENCE_RECH,nombre de fois que le client a fait une recharge,number of times the customer refilled
7,REVENUE,revenu mensuel de chaque client,monthly income of each client
8,ARPU_SEGMENT,revenu sur 90 jours/3,income over 90 days / 3
9,FREQUENCE,nombre de fois que client à fait un revenu,number of times the client has made an income


In [ ]:
#loading the dataset
train=pd.read_csv(path+'Train.csv')

In [ ]:
train.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


The train data takes quite some times to run because of its extremely large nature, before any further analysis,let's try to reduce the memory allocated to the data size using the function below.

In [ ]:
def reduce_mem_usage(train_data):
    """ iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.

    it's a customize variable. 
    
    Here the train_data input is a datframe (train or test)
    """
    start_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in train_data.columns:
        col_type = train_data[col].dtype

        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train_data[col] = train_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('category')

    end_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return train_data

In [ ]:
#to use the function
import gc

train = reduce_mem_usage(train)
gc.collect()

Memory usage of dataframe is 312.25 MB
Memory usage after optimization is: 184.77 MB
Decreased by 40.8%


88

In [ ]:
#same with the test data
test=pd.read_csv(path+'Test.csv')

In [ ]:
test=reduce_mem_usage(test)
gc.collect()

Memory usage of dataframe is 52.20 MB
Memory usage after optimization is: 27.77 MB
Decreased by 46.8%


249

# Expresso Churn EDA

We start by cleaning our data may be merging the train and test when performing the analysis will be nice 



In [ ]:
train.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0


In [ ]:
train.CHURN.value_counts() #quite imbalance, if we want to train like this

0    1750062
1     403986
Name: CHURN, dtype: int64

In [ ]:
data=pd.concat([train,test]).reset_index(drop=True) #merging the data

In [ ]:
data.head()

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,1.0,2.0,NO,54,On net 200F=Unlimited _call24H,8.0,0.0
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,4,NaN,NaN,1.0
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,NaN,NaN,NO,17,On-net 1000F=10MilF;10d,1.0,0.0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,NaN,NaN,NO,62,"Data:1000F=5GB,7d",11.0,0.0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,NaN,NaN,NO,11,Mixt 250F=Unlimited_call24H,2.0,0.0


In [ ]:
data['REGION']=data['REGION'].astype('object')

In [ ]:
#converting a category type variable to object
data['TOP_PACK']=data['TOP_PACK'].astype('object')

In [ ]:
### converting into categorical variable
def tenure_cat(data):
  cond=[(data.TENURE=='K > 24 month'),(data.TENURE=='I 18-21 month'),(data.TENURE=='H 15-18 month'),(data.TENURE=='G 12-15 month'),(data.TENURE=='J 21-24 month'),(data.TENURE=='F 9-12 month'),(data.TENURE=='E 6-9 month'),(data.TENURE=='D 3-6 month')]
  val=[24,18,15,12,21,9,6,3]
  data['TENURE']=np.select(cond,val)

  return data

In [ ]:
data=tenure_cat(data)

## Preprocessing

In [ ]:
def preprocess(data):

  ##filling missing values for categorical

  data.REGION.fillna('region',inplace=True)

  data.TOP_PACK.fillna('pack',inplace=True)

  ## dealing with missing values for numerical variables

  columns=[col for col in data.columns if col not in ['user_id','REGION','TENURE','CHURN','TOP_PACK','MRG']]

  for col in columns:
    data[col].fillna(-9999,inplace=True)
  ###top_up_per_month######

  data['top_up_per_month']=data['MONTANT']/data['FREQ_TOP_PACK']

  ### tenure_per_regularity #######

  data['tenure_per_reg']=data['TENURE']/data['REGULARITY']

  data['income_to_amount']=data['REVENUE']/data['MONTANT']

  data['tenure_freq_rech']=data['REVENUE']/data['FREQUENCE_RECH']

  data['income_90_days_per_revenue']=data['ARPU_SEGMENT']/data['REVENUE']

  data['revenue_after_topup']=data['REVENUE']-data['MONTANT']

  data=data.drop(columns=['MRG'])

  _region=pd.get_dummies(data['REGION'])

  _pack=pd.get_dummies(data['TOP_PACK'])

  data=pd.concat([data.drop('REGION',axis=1),_region],axis=1)

  data=pd.concat([data.drop('TOP_PACK',axis=1),_pack],axis=1)

  ## target encoding?---we will discuss about that

  ## Principal component analysis? ----(class_1,class_2,class_4) this is feature engineering

  ## Discuss about frequency encoding ? 

  ## Building our model--- (K-fold cross validation, stratified k-fold cv)

  ## Build an ensemble model-- (LGBM+XGBoost+RandomForest)

  ## Post-processing--distribution of the target is discrete (since they are classes)
     #SO we can improve the performance by creating a range of values in a discrete interval

  return data 


In [ ]:
## preprocessing the data 
data=preprocess(data)